Qual perfil dos sintomas das pessoas que pegam Covid?

In [632]:
import pandas as pd
import matplotlib
matplotlib.use('qt5Agg') # case does not matter tkaGg, TkAgg will do
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tkinter
import re

import holoviews as hv
import hvplot.pandas

import panel as pn
pn.extension()

from vega_datasets import data as vds

import cartopy
import cartopy.feature as cf
from cartopy import crs as ccrs

import geocoder

import json
import re
import json
import glob
import datetime


In [633]:
# Gerando os URL dos .csv
caminho = r'dados/dados-sp-'
todos_dados_csv = glob.glob(caminho + "*.csv")

# Colunas a serem lidas
cols = ["sintomas", "evolucaoCaso", "classificacaoFinal"]

# Carregando os dados
dados = pd.concat((pd.read_csv(f, sep=';', encoding='latin1', usecols=cols, infer_datetime_format = False) for f in todos_dados_csv), ignore_index = True)

In [634]:
# Exclui as linhas cujas colunas indicadas abaixo sao null
dados.dropna(subset = ["classificacaoFinal"], inplace = True)

# Exclui os dados Cancelados
dados.drop(dados[dados['evolucaoCaso'] == 'Cancelado'].index, inplace = True)

dados.reset_index(drop=True, inplace = True) # Realoca os indices depois das exclusoes

In [635]:
sintomas = dados["sintomas"]
classificacaoFinal = dados["classificacaoFinal"]
evolucaoDoCaso = dados["evolucaoCaso"]

In [636]:
dadosTeste = []
teste4 = ''
string1 = ['Assintomático']
string2 = ['Outros: Paciente assintomático']
string3 = ['Paciente assintomático']


##Alem de separar as palavras para criar uma dados de Sintomas, ainda conta quantos sintomas cada pessoa teve
for i in range(len(sintomas)):
    teste = []
    if(type(sintomas[i]) == str):
        teste.append(re.split(', |,, |,|,,', sintomas[i]))
    else:
        teste.append(teste4)
    
    
    if(teste[0] == string1 or teste[0] == string2 or teste[0] == string3):
        teste[0] = ['Assintomático']
        teste.append(0)
    else:
        if(len(teste[0]) == 10):
            continue
        teste.append(len(teste[0]))
    teste.append(classificacaoFinal[i])
    teste.append(evolucaoDoCaso[i])
    dadosTeste.append(teste)

In [637]:

dadosPanda = pd.DataFrame(dadosTeste)
dadosPandaCovid = pd.DataFrame(dadosTeste)


dadosPanda.rename(columns={0:'sintomas',1:'numSintomasPorPessoas' ,2:'classificacaoFinal', 3:'evolucaoCaso'},inplace = True)
dadosPandaCovid.rename(columns={0:'sintomas',1:'numSintomasPorPessoas' ,2:'classificacaoFinal', 3:'evolucaoCaso'},inplace = True)

In [638]:
dadosPandaCovid.drop(dadosPandaCovid[dadosPandaCovid['classificacaoFinal'] == 'Descartado'].index, inplace=True)
dadosPandaCovid.drop(dadosPandaCovid[dadosPandaCovid['classificacaoFinal'] == 'Síndrome Gripal Não Especificada'].index, inplace=True)



dadosPanda.reset_index(drop=True, inplace = True)
dadosPandaCovid.reset_index(drop=True, inplace = True)

In [669]:
dadosPandaSemCovidComGripe = dadosPanda.loc[(dadosPanda['classificacaoFinal'] == 'Síndrome Gripal Não Especificada')]
dadosPandaSemCovidSemGripe = dadosPanda.loc[(dadosPanda['classificacaoFinal'] == 'Descartado')]

dadosPandaSemCovidComGripe.reset_index(drop=True, inplace = True)
dadosPandaSemCovidSemGripe.reset_index(drop=True, inplace = True)


,sintomas,numSintomasPorPessoas,classificacaoFinal,evolucaoCaso
0,"[Outros, Febre]",2,Descartado,Cura
1,[Outros],1,Descartado,Cura
2,[Tosse],1,Descartado,Cura
3,[Assintomático],0,Descartado,Cura
4,"[Febre, Tosse]",2,Descartado,Cura
...,...,...,...,...
2232649,[Tosse],1,Descartado,Cura
2232650,"[Tosse, Outros]",2,Descartado,Cura
2232651,[Assintomático],0,Descartado,Cura
2232652,"[Dispneia, Tosse]",2,Descartado,Ignorado


In [640]:
matrizPandaCovid

,sintomas,numSintomasPorPessoas,classificacaoFinal,evolucaoCaso
0,[Dor de Cabeça],1,Confirmado Laboratorial,Cura
1,[Outros],1,Confirmado Laboratorial,Em tratamento domiciliar
2,[Outros],1,Confirmado Laboratorial,Cura
3,"[Febre, Outros, Dor de Garganta]",3,Confirmado Laboratorial,Cura
4,"[Febre, Tosse, Outros]",3,Confirmado Laboratorial,Em tratamento domiciliar
...,...,...,...,...
1892926,"[Febre, Dor de Garganta, Dor de Cabeça, Distúr...",5,Confirmado Laboratorial,Cura
1892927,[Tosse],1,Confirmado Laboratorial,Cura
1892928,"[Febre, Outros]",2,Confirmado Laboratorial,Cura
1892929,[Assintomático],0,Confirmado Laboratorial,Cura


In [641]:
sintomasPorPessoaTotal = dadosPanda.numSintomasPorPessoas.value_counts()
sintomasPorPessoaCovid = dadosPandaCovid.numSintomasPorPessoas.value_counts()

Vamos calcular a Probabilidade Condicional de numeros de sintomas e pessoas que pegaram covid

In [642]:
pB = sintomasPorPessoaTotal.sort_index()/sintomasPorPessoaTotal.sum()

P(B) = Probabilidade da pessoa ter "B" sintomas


In [643]:
pA = matrizPandaCovid['classificacaoFinal'].count()/matrizPanda['classificacaoFinal'].count()

P(A) = probabilidade das pessoas com covid confirmado 

In [683]:
pAinterseccaoB = sintomasPorPessoaCovid.sort_index()/sintomasPorPessoaTotal.sum()

P(A intersecçao B)

In [681]:
pBcondicionalA = pAinterseccaoB/pB

for key in pBcondicionalA.keys():
    pBcondicionalA[key] = round(pBcondicionalA[key]*100, 2)

%matplotlib
plt.bar(pBcondicionalA.keys() , pBcondicionalA.values)

plt.xlabel("Número de sintomas")
plt.ylabel("Porcentagem de chance de ser COVID-19")
plt.title('Porcentagem de chance de ser COVID-19 a partir da quantidade de sintomas')


plt.rcParams.update({'font.size': 20})

plt.show()

Using matplotlib backend: Qt5Agg


In [679]:
data = [dadosPandaCovid['numSintomasPorPessoas'], dadosPandaSemCovidSemGripe['numSintomasPorPessoas']]

fig7, ax7 = plt.subplots()

ax7.boxplot(data)

teste = ['Pessoas com COVID-19' , 'Pessoas sem COVID-19']

plt.xticks([1, 2],teste)
plt.ylabel("Número de sintomas")
plt.title('BoxPlot do número de sintomas')

Text(0.5, 1.0, 'BoxPlot do número de sintomas')

P(B/A) = Probabilidade da pessoa ter covid a partir da quantidade de sintomas que ela possui

A maior probabilidade da pessoa ter covid é quando ela tem 9 Sintomas, que é 60.83% 

In [647]:
def contarPalavras(dados):
    contadorPalavrasCovid = {}
    for linhas in dados['sintomas']:
        for palavra in linhas:
            if(palavra == 'Distúrbios Gustativos'):
                palavra = 'Gustativos'
            elif(palavra == 'Distúrbios Olfativos'):
                palavra = 'Olfativos'
            elif(palavra == 'Dor De Garganta'):
                palavra =  'Dor de Garganta'
            elif(palavra == ''):
                continue
            elif(palavra == 'Dispnéia'):
                palavra = 'Dispneia'
            contadorPalavrasCovid.setdefault(palavra, 0)
            contadorPalavrasCovid[palavra] += 1

    return contadorPalavrasCovid


Chance de desenvolver um sintoma quando tiver covid

Por ser uma distribuição de Bernoulli, a frequencia relativa dada a uma população relativamente grande, também é seu estimador 

In [648]:
contadorPalavrasCovid = contarPalavras(matrizPandaCovid)
sintomasCovid = contadorPalavrasCovid.copy()

for key in contadorPalavrasCovid:
    contadorPalavrasCovid[key] = contadorPalavrasCovid[key]/len(matrizPandaCovid)

Aqui mostra a chance de voce ter um sintoma se voce tem covid

In [690]:
%matplotlib

plt.bar(sorted(contadorPalavrasCovid.keys()), sorted(contadorPalavrasCovid.values()))
plt.xlabel("sintomas")
plt.ylabel("Frequência Relativa")
plt.title('Frequência Relativa dos sintomas para as pessoas que tiveram COVID-19')

plt.rcParams['xtick.labelsize'] = 12
plt.show()


Using matplotlib backend: Qt5Agg


Agora vamos fazer a mesma análise dos sintomas para obitos e para cura 

In [650]:
contadorPalavrasTotal = contarPalavras(matrizPanda)

sintomasTotal = contadorPalavrasTotal.copy()

In [651]:
dadosPandaCovidObitos = dadosPandaCovid.copy()

dadosPandaCovidObitos.drop(dadosPandaCovidObitos[dadosPandaCovidObitos['evolucaoCaso'] != 'Óbito'].index, inplace=True)
dadosPandaCovidObitos.reset_index(drop=True, inplace = True)

In [715]:
contadorPalavrasCovidObitos = contarPalavras(dadosPandaCovidObitos)

sintomasCovidObitos = contadorPalavrasCovid.copy()

for key in contadorPalavrasCovidObitos:
    contadorPalavrasCovidObitos[key] = contadorPalavrasCovidObitos[key]/sintomasCovid[key]

In [719]:
plt.bar(sorted(contadorPalavrasCovidObitos.keys()), sorted(contadorPalavrasCovidObitos.values()), color='red')

plt.xlabel("sintomas")
plt.ylabel("Frequência Relativa")
plt.title('Frequência de óbitos de covid para cada sintoma')

plt.show()

In [654]:
dadosPandaCura = dadosPandaCovid.copy()

dadosPandaCura.drop(dadosPandaCura[dadosPandaCura['evolucaoCaso'] != 'Cura'].index, inplace=True)
dadosPandaCura.reset_index(drop=True, inplace = True)

In [711]:
contadorPalavrasCura = contarPalavras(dadosPandaCura)

sintomasCura = contadorPalavrasCura.copy()

for key in contadorPalavrasCura:
    contadorPalavrasCura[key] = contadorPalavrasCura[key]/sintomasCovid[key]

In [714]:
plt.bar(sorted(contadorPalavrasCura.keys()), sorted(contadorPalavrasCura.values()), color='green')

plt.xlabel("sintomas")
plt.ylabel("Frequência Relativa")
plt.title('Frequência de curados para cada sintoma')

plt.show()

In [706]:

fig, ax = plt.subplots()

ind = np.arange(10)
width = 0.35

rects1 = ax.bar(ind, sorted(contadorPalavrasCura.values()), width, color='g')
rects2 = ax.bar(ind + width, sorted(contadorPalavrasCovidObitos.values()), width, color='r')

ax.set_xticks(ind + width/2)
ax.set_xticklabels(('Assintomático', 'Coriza', 'Dispneia', 'Gustativos', 'Olftativos', 'Dor de Cabeça', 'Dor de garganta', 'Febre', 'Outros', 'Tosse'))

ax.legend((rects1[0], rects2[0]), ('Cura', 'Óbitos'))

plt.show()


Correlação entre os sintomas

Vamos pegar os Três sintomas mais frequentes (tosse, febre e Dor de cabeça) e verificar se eles possuem alguma correlação 

In [722]:
numFebreTosse = 0
numFebreDorDeCabeca = 0
numDorDeCabecaTosse = 0

for linha in dadosPandaCovid['sintomas']:
    if(('Febre' in linha) and ('Tosse' in linha)):
        numFebreTosse += 1
    elif(('Febre' in linha) and ('Dor de Cabeça' in linha)):
        numFebreDorDeCabeca += 1
    elif(('Tosse' in linha) and ('Dor de Cabeça' in linha)):
        numDorDeCabecaTosse += 1

print('A quantidade de vezes que apareceu os sintomas ao mesmo momento foi')

freqFebreTosse = round((numFebreTosse+numFebreDorDeCabecaTosse)/sintomasCovid['Tosse'], 2)
freqTosseFebre = round((numFebreTosse+numFebreDorDeCabecaTosse)/sintomasCovid['Febre'], 2)
freqFebreDorDeCabeca = round((numFebreDorDeCabeca+numFebreDorDeCabecaTosse)/sintomasCovid['Dor de Cabeça'],2)
freqDorDeCabecaFebre = round((numFebreDorDeCabeca+numFebreDorDeCabecaTosse)/sintomasCovid['Febre'],2)
freqDorDeCabecaTosse = round((numDorDeCabecaTosse+numFebreDorDeCabecaTosse)/sintomasCovid['Tosse'],2)
freqTosseDorDeCabeca = round((numDorDeCabecaTosse+numFebreDorDeCabecaTosse)/sintomasCovid['Dor de Cabeça'],2)


print('A febre apareceu em ', freqFebreTosse, ' vezes que tosse apareceu e a tosse apareceu em ', freqTosseFebre , 'vezes que a febre apareceu')
print('A febre apareceu em ', freqFebreDorDeCabeca , ' vezes que Dor de Cabeca apareceu e a Dor de Cabeca apareceu em ', freqDorDeCabecaFebre , 'vezes que a febre apareceu')
print('A Dor de Cabeça apareceu em ',freqDorDeCabecaTosse, ' vezes que Tosse apareceu e a Tosse apareceu em ', freqTosseDorDeCabeca, 'vezes que a Dor de cabeça apareceu')



A quantidade de vezes que apareceu os sintomas ao mesmo momento foi
A febre apareceu em  0.59  vezes que tosse apareceu e a tosse apareceu em  0.86 vezes que a febre apareceu
A febre apareceu em  0.4  vezes que Dor de Cabeca apareceu e a Dor de Cabeca apareceu em  0.37 vezes que a febre apareceu
A Dor de Cabeça apareceu em  0.36  vezes que Tosse apareceu e a Tosse apareceu em  0.56 vezes que a Dor de cabeça apareceu


Com isso, é possível perceber que o sintoma que mais aparece relacionado com os outros sintomas é a Tosse que aparece muito em conjunto com dor de cabeça ou febre mas não aparecem muito esses 3 sintomas juntos.

In [726]:
listaDeFreqY = [freqFebreTosse,
                     freqTosseFebre,
                     freqFebreDorDeCabeca,
                     freqDorDeCabecaFebre,
                     freqDorDeCabecaTosse,
                     freqTosseDorDeCabeca,
                     ]

listaDeFreqX = ['Febre|Tosse', 
                'Tosse|Febre',
                'Febre|Dor de Cabeca ',
                'Dor de Cabeca|febre',
                'Dor de Cabeca|tosse',
                'Tosse|dor de cabeca',
                ]

plt.bar(listaDeFreqX, listaDeFreqY)

plt.xlabel("sintomas")
plt.ylabel("Frequência Relativa")
plt.title('Probabilidade condicional entre os sintomas')

plt.rcParams['xtick.labelsize'] = 12
plt.show()